In [ ]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from PIL import Image
import pickle
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision import transforms, models
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
from torch.cuda.amp import GradScaler, autocast

# Disable the DecompressionBombError by setting the MAX_IMAGE_PIXELS to None
Image.MAX_IMAGE_PIXELS = None

# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the path to the .npz file
npz_file_path = path/to/your/npz/file'
'In this project the npz files were very usefull but ofcource it can be the files that the images are stored directly here'

# Check if the .npz file exists
if not os.path.exists(npz_file_path):
    print("NPZ file not found.")
    exit()  # Exit the program if the file is missing

# Load the .npz file
data = np.load(npz_file_path, allow_pickle=True)

# Retrieve datasets
X_train = data['X_train']
y_train = data['y_train']
X_val = data['X_val']
y_val = data['y_val']
X_test = data['X_test']
y_test = data['y_test']

print(f"Number of training samples: {len(X_train)}")
print(f"Number of validation samples: {len(X_val)}")
print(f"Number of test samples: {len(X_test)}")

# Convert file paths from numpy arrays to lists
X_train = X_train.tolist()
X_val = X_val.tolist()
X_test = X_test.tolist()


class CustomResizeAndPadOrCrop:
    def __init__(self, target_size):
        self.target_size = target_size

    def __call__(self, img):
        width, height = img.size

        # Check if the image needs to be resized and padded or cropped
        if width < self.target_size[0] or height < self.target_size[1]:
            # Calculate padding to center the image
            padding_width = max((self.target_size[0] - width) // 2, 0)
            padding_height = max((self.target_size[1] - height) // 2, 0)

            # Create a new blank RGB image of target size
            padded_img = Image.new("L", self.target_size, color=0)  # Set color to black for RGB

            # Paste the original image onto the blank image at the center
            padded_img.paste(img, (padding_width, padding_height))

            # Return the padded image
            img = padded_img

        elif width > self.target_size[0] or height > self.target_size[1]:
            # Crop the image from the top-left corner
            img = transforms.functional.crop(img, 0, 0, self.target_size[1], self.target_size[0])

        return img


class EarlyStopping:
    def __init__(self, patience=25, verbose=True):
        self.patience = patience
        self.verbose = verbose  # Set verbose to True to enable printing
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf

    def __call__(self, val_loss, model):
        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

        return self.early_stop

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decreases.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), 'checkpoint.pt')
        self.val_loss_min = val_loss


# Define transformations
transform = transforms.Compose([
    CustomResizeAndPadOrCrop((1024, 1024)),  # Adjust size as needed
    transforms.ToTensor(),
    #transforms.Normalize(mean=[0.485, 0.456, 0.406],  
    # Normalize with ImageNet mean
    # std=[0.229, 0.224, 0.225])  Normally this transformation help , although it didn't this time
])

# Custom Dataset Class
class ImageDataset(Dataset):
    def __init__(self, file_paths, labels, transform=None):
        self.file_paths = file_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        image = Image.open(self.file_paths[idx])
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

# Create datasets and dataloaders
batch_size = 32
train_dataset = ImageDataset(X_train, y_train, transform=transform)
val_dataset = ImageDataset(X_val, y_val, transform=transform)
test_dataset = ImageDataset(X_test, y_test, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Define a Simple CNN Model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=15, kernel_size=5)
        self.pool1 = nn.MaxPool2d(kernel_size=4, stride=4)
        self.conv2 = nn.Conv2d(in_channels=15, out_channels=25, kernel_size=5)
        self.pool2 = nn.MaxPool2d(kernel_size=4, stride=4)
        self.fc1 = nn.Linear(in_features=25 * 62 * 62, out_features=1000)
        self.fc2 = nn.Linear(in_features=1000, out_features=2)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = x.view(-1, 25 * 62 * 62)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Instantiate and move model to GPU
model = SimpleCNN().to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adagrad(model.parameters(), lr=0.001)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=2)

early_stopping = EarlyStopping(patience=10, verbose=True)

# Training Loop
# Train the model
epochs = 50
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    # Validate the model
    model.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels.long())
            val_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    val_loss = val_loss / len(val_dataset)
    train_acc = correct_train / total_train
    val_acc = correct_val / total_val

    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {epoch_loss:.4f}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

    # Adjust learning rate based on validation loss
    scheduler.step(val_loss)

    # Early stopping check
    if early_stopping(val_loss, model):
        print("Early stopping triggered.")
        break

# Test the model
model.eval()
test_loss = 0.0
test_correct = 0
test_total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, labels.long())
        test_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

test_loss /= len(test_dataset)
test_acc = test_correct / test_total

print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}")

# Calculate Precision, Recall, and F1 Score
test_predictions = []
test_targets = []
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        test_predictions.extend(predicted.cpu().tolist())
        test_targets.extend(labels.cpu().tolist())

precision = precision_score(test_targets, test_predictions)
recall = recall_score(test_targets, test_predictions)
f1 = f1_score(test_targets, test_predictions)
print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

# Compute and print the confusion matrix
cm = confusion_matrix(test_targets, test_predictions)
print("\nConfusion Matrix:")
print("          Predicted")
print("         0      1")
print(f"Actual 0 {cm[0,0]}    {cm[0,1]}")
print(f"Actual 1 {cm[1,0]}    {cm[1,1]}")

# Save the trained model
save_path = 'path/to/save'  #here put the path you want to save your mode
torch.save(model.state_dict(), save_path)
print(f"Model saved at {save_path}")
